# ARIS/ARIL Working File

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly" # you may have to comment this out if plotly isn't installed

# 2024-04-10

## Import of Data from CSV Files

In [ ]:
# Case Picking Data
df_OrderData_CasePick = pd.read_csv("ARIS_ARIL_Data/CasePicking/orderlines.csv", sep=";") # set path and file name 
df_MasterData_CasePick = pd.read_csv("ARIS_ARIL_Data/CasePicking/itemmaster.csv", sep=";") # set path and file name 

# e-com Data
data_path_eCom = 'ARIS_ARIL_Data/e-com/' # set path and file name 
File_ItemMaster_eCom = 'item_master.csv' # set path and file name 
File_OrderData_eCom = 'orderline_data.csv' # set path and file name 

df_MasterData_eCom = pd.read_csv(data_path_eCom+File_ItemMaster_eCom, sep='\t')
df_OrderData_eCom = pd.read_csv(data_path_eCom+File_OrderData_eCom, sep='\t', parse_dates = ['orderTime']) # here we parse the column 'orderTime' directly to DateTime format

## Defining Column Names
Defining variables for column names allows more convenient access to columns, especially when using column names as arguments in functions.

In [ ]:
col_length = 'depth'
col_width = 'width'
col_height = 'height'
col_weight = 'weight'

col_ItemID = 'itemID'
col_OrderID = 'orderID'

col_OrderDate = "OrderDate"
col_OrderWeekDay = "OrderWeekDay"
col_OrderHour = "OrderHour"

## Preparing the Data Before Use

### Case Picking Data

In [ ]:
# Adjusting Data Types
df_OrderData_CasePick["orderID"] = df_OrderData_CasePick["orderID"].astype(str)
df_OrderData_CasePick["customerID"] = df_OrderData_CasePick["customerID"].astype(str)
df_OrderData_CasePick["orderTime"] = pd.to_datetime(df_OrderData_CasePick["orderTime"])

### e-com Data

In [ ]:


df_OrderData_eCom[col_OrderID] = df_OrderData_eCom[col_OrderID].astype(str)
df_OrderData_eCom[col_OrderDate] = df_OrderData_eCom['orderTime'].dt.date
df_OrderData_eCom[col_OrderWeekDay] = df_OrderData_eCom['orderTime'].dt.day_name()
df_OrderData_eCom[col_OrderHour] = df_OrderData_eCom['orderTime'].dt.hour


In [ ]:
df_OrderData_CasePick

## Descriptive Analyses

### SKU Count

In [ ]:
df_MasterData_CasePick["itemID"].nunique()
df_OrderData_CasePick["itemID"].nunique()

### Observation Period

In [ ]:
df_OrderData_CasePick['orderTime'].nunique()

## Cleaning the Data
### Case Picking Data

In [ ]:
# This is the code we took from ChatGPT
# Replace 0 values with NaN in the specified columns
df_MasterData_CasePick[['depth', 'width', 'height', 'weight']] = df_MasterData_CasePick[['depth', 'width', 'height', 'weight']].replace(0, pd.NA)

# Filter out rows where any of the specified columns have NaN values
df_MasterData_CasePick = df_MasterData_CasePick.dropna(subset=['depth', 'width', 'height', 'weight'])

# Optionally, you can also filter out nonsensical values based on some threshold
# For example, if you consider negative values as wrong for dimensions:
df_MasterData_CasePick = df_MasterData_CasePick[(df_MasterData_CasePick['depth'] > 0) & (df_MasterData_CasePick['width'] > 0) & (df_MasterData_CasePick['height'] > 0) & (df_MasterData_CasePick['weight'] > 0)]

# Now df_MasterData_CasePick contains only rows where length, width, height, and weight are not 0, empty, or nonsensical

# If you want to reset the index of the filtered DataFrame
df_MasterData_CasePick.reset_index(drop=True, inplace=True)

# Now df_MasterData_CasePick is your DataFrame with the desired filtering applied


### Visualizing Dimensions in Item Master to Detect Implausible Entries

In [ ]:
df_MasterData_CasePick[['depth', 'width', 'height']].describe()

In [ ]:

#df_MasterData_CasePick[['depth', 'width', 'height']].plot.box()
df_MasterData_eCom[['depth', 'width', 'height']].plot.box()


In [ ]:
df_MasterData_CasePick[[col_ItemID,'depth', 'width', 'height']].sort_values(by='height', ascending=False)
df_MasterData_CasePick_Ugly = df_MasterData_CasePick[df_MasterData_CasePick['height'] > 1000] # Note: Here we are only checking for height. We should include the other dimensions, too. Please try this yourself.
df_MasterData_CasePick_Ugly



#### Checking how many orderlines are created with items with extreme dimension

In [ ]:
df_OrderData_CasePick_Ugly = pd.merge(df_OrderData_CasePick, df_MasterData_CasePick_Ugly, on=col_ItemID, how='inner')

In [ ]:
df_OLpD_CasePick_Ugly = df_OrderData_CasePick_Ugly.groupby('orderTime').size().reset_index(name='Count_OL')
df_OLpD_CasePick_Ugly

## Time Profiles

### e-com Data

In [ ]:
# Orders per Day
df_OpD_eCom = df_OrderData_eCom.groupby(col_OrderDate)[col_OrderID].nunique().reset_index(name='OpD')
df_OpD_eCom.plot.bar(x=col_OrderDate, y='OpD')

In [ ]:
# Orderlines per Day
df_OLpD = df_OrderData_eCom.groupby(col_OrderDate).size().reset_index(name='OLpD')
df_OLpD
#df_OLpD.plot.bar(x=col_OrderDate, y='OLpD')

In [ ]:
# SKUs per Day
df_SKUspD = df_OrderData_eCom.groupby(col_OrderDate)[col_ItemID].nunique().reset_index(name='SKUs_p_D')
df_SKUspD

In [ ]:
df_SKUs_pH_eCom = df_OrderData_eCom.groupby([col_OrderWeekDay,col_OrderHour])[col_ItemID].nunique().reset_index()
df_SKUs_pH_eCom

In [ ]:
df_OpWeekDay_eCom = df_OrderData_eCom.groupby(col_OrderWeekDay)[col_OrderID].nunique().reset_index()
df_OpWeekDay_eCom.plot.bar(x=col_OrderWeekDay, y=col_OrderID)

## Order Structure

### e-com Data

In [ ]:
# e-com data
df_OL_p_O_eCom = df_OrderData_eCom.groupby(col_OrderID).size().reset_index(name='OL_p_O')


#### Identifying Orders with "Many" orderlines (> n)

In [ ]:
n = 50
df_OL_p_O_High = df_OL_p_O_eCom[df_OL_p_O_eCom['OL_p_O'] >= 50]
df_OL_p_O_High

In [ ]:
df_OL_p_O_eCom['OL_p_O'].plot.box()

In [ ]:
df_OrderData_eCom_LargeOrders = pd.merge(df_OL_p_O_High, df_OrderData_eCom, on=col_OrderID, how='inner')

In [ ]:
df_OrderData_eCom_LargeOrders

In [ ]:
df_OrderData_eCom_LargeOrders.groupby(col_OrderDate)[col_OrderID].nunique().plot.bar()

#### Identifying Orderlines with High Quantities

In [ ]:
df_OrderData_eCom_LargeQty = df_OrderData_eCom[df_OrderData_eCom['qty']>50].groupby(col_OrderDate)['qty'].size().reset_index()

In [ ]:
df_OrderData_eCom_LargeQty.plot.bar(x=col_OrderDate, y='qty')